In [1]:
import pandas as pd

df = pd.read_csv("finalData.csv")
df = df.drop(columns={"Unnamed: 0"})

In [2]:
df.head()

,datetime,energy_consumed_MW,hour_of_day,day_of_week,month,is_weekend,lag_1_day,lag_7_day,lag_14_day,lag_1_hour,...,roll_mean_1_day,roll_mean_7_day,roll_std_7_day,roll_mean_14_day,roll_std_14_day,roll_mean_1_hour,roll_mean_2_hour,roll_std_2_hour,roll_mean_24_hour,roll_std_24_hour
0,1998-04-03 01:00:00,20905.0,1,4,4,False,22795.0,28891.0,30140.0,22795.0,...,22795.0,27843.714286,2791.129563,28861.500000,2188.247235,22795.0,24072.5,1806.657826,26587.333333,4081.905572
1,1998-04-03 02:00:00,19974.0,2,4,4,False,20905.0,29438.0,30017.0,20905.0,...,20905.0,26702.857143,3756.782508,28201.857143,3010.610285,20905.0,21850.0,1336.431816,26571.250000,4104.379837
2,1998-04-03 03:00:00,19215.0,3,4,4,False,19974.0,30656.0,30118.0,19974.0,...,19974.0,25350.857143,4275.544271,27484.500000,3669.285589,19974.0,20439.5,658.316413,26557.583333,4126.648329
3,1998-04-03 04:00:00,19488.0,4,4,4,False,19215.0,29696.0,30175.0,19215.0,...,19215.0,23716.428571,4092.399005,26705.714286,4187.763722,19215.0,19594.5,536.694047,26533.125000,4169.941479
4,1998-04-03 05:00:00,19928.0,5,4,4,False,19488.0,28080.0,29989.0,19488.0,...,19488.0,22258.142857,3359.675849,25942.357143,4471.165768,19488.0,19351.5,193.040151,26525.333333,4183.466034


In [3]:
df.columns

Index(['datetime', 'energy_consumed_MW', 'hour_of_day', 'day_of_week', 'month',
       'is_weekend', 'lag_1_day', 'lag_7_day', 'lag_14_day', 'lag_1_hour',
       'lag_2_hour', 'lag_24_hour', 'roll_mean_1_day', 'roll_mean_7_day',
       'roll_std_7_day', 'roll_mean_14_day', 'roll_std_14_day',
       'roll_mean_1_hour', 'roll_mean_2_hour', 'roll_std_2_hour',
       'roll_mean_24_hour', 'roll_std_24_hour'],
      dtype='object')

In [4]:
required_cols = ["datetime", "energy_consumed_MW", "day_of_week", "month", "is_weekend", "lag_1_day", "lag_7_day", "lag_14_day", "roll_mean_1_day", "roll_mean_7_day", "roll_mean_14_day", "roll_std_7_day", "roll_std_14_day"]

df_new = df[required_cols].copy()

df_new['datetime'] = df_new['datetime']
df_new['datetime'] = pd.to_datetime(df_new['datetime'])
df_new = df_new.sort_values("datetime").reset_index(drop=True)

In [5]:
# create target before splitting
df_new["target"] = df_new["energy_consumed_MW"].shift(-1)

# drop last row since it has NaN target
df_new = df_new.dropna(subset=["target"])

# split chronologically
split_date = df_new["datetime"].max() - pd.Timedelta(days=365)
train = df_new[df_new["datetime"] < split_date].copy()
test = df_new[df_new["datetime"] >= split_date].copy()

print("Train:", train["datetime"].min(), "→", train["datetime"].max())
print("Test:", test["datetime"].min(), "→", test["datetime"].max())

X_train = train.drop(["energy_consumed_MW", "datetime", "target"], axis=1)
y_train = train["target"]

X_test = test.drop(["energy_consumed_MW", "datetime", "target"], axis=1)
y_test = test["target"]

Train: 1998-04-03 01:00:00 → 2017-08-02 23:00:00
Test: 2017-08-03 00:00:00 → 2018-08-03 00:00:00


In [ ]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from xgboost import XGBRegressor
import joblib


def train_evaluate_save(model, name, filename):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)
    rmse = root_mean_squared_error(y_test, preds)
    
    print(f"\n📊 {name}")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    
    # Save model
    joblib.dump(model, f"models/{filename}")
    print(f"✅ Saved {name} model to {filename}")
    
    return model

In [7]:
# 1. XGBoost
xgb = train_evaluate_save(
    XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=6, n_jobs=-1,
                 objective="reg:squarederror", random_state=42),
    "XGBoost", "xgboost_model.pkl"
)


📊 XGBoost
MAE: 6708.50
RMSE: 8875.96
✅ Saved XGBoost model to xgboost_model.pkl
